# Step13 가변 길이 인수(역전파 편)
## 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

In [1]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

In [13]:
class Function:
  '''
    가변 길이로 받은 inputs들을 unpack하고, 출력이 tuple 형태가 아닐 경우엔 추가 지원
  '''
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    
    for output in outputs:
      output.set_creator(self) 
    
    self.inputs = inputs
    self.outputs= outputs 
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

class Add(Function):
  '''
    사용하기 쉬운 직관적 형태로 변환!
  '''
  def forward(self, x0, x1):
    y = x0 + x1
    return y 

In [14]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    '''
      backward 부분 추가
    '''
    return gy, gy

def add(x0, x1):
  return Add()(x0, x1)

In [15]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    '''
      여러개의 입출력을 받을 수 있도록 준비.
    '''
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      gys = [output.grad for output in f.outputs] # output의 grad 가져옴
      gxs = f.backward(*gys)                      # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        x.grad = gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          funcs.append(x.creator) 

In [16]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data # Function 함수에서 inputs (튜플)로 받기 때문에 input-> inputs[0]으로 바꿔줌
    gx = 2 * x * gy
    return gx

def square(x):
  return Square()(x)

In [17]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0
